# Create list of all oncology trials from ClinicalTrials.gov api closest to the Subject

In [ ]:
install.packages("janitor")
library(SparkR)
library(readr)
library(janitor)
library(httr2)
library(purrr)
library(dplyr)
library(glue)
library(readr)


In [ ]:
get_studies <- function(usa_zipcode_csv,zipcode,radius,indication){

        usa_zipcodes <- read_csv(usa_zipcode_csv) # List of zipcodes with geo location across USA

        # Base URL for the API call
        base_url <- "https://clinicaltrials.gov/api/v2/studies"
        
        #Get geo location with lat and lon from zipcode from user 
        zip_lat_lon <- dplyr::filter(usa_zipcodes,zip==zipcode)
        geo <- paste(zip_lat_lon$latitude,zip_lat_lon$longitude,paste0(radius,"mi"),sep = ",")
        print(paste("Geo loc:", geo))

        # Initialize the payload without the skip parameter
        payload <- list(
        format = "json",
        markupFormat = "markdown",
        `query.cond` = indication,
        `filter.overallStatus` = "NOT_YET_RECRUITING|RECRUITING",
        `filter.geo` = glue("distance(",geo,")"),
        fields = "NCTId",
        countTotal = "true",
        pageSize = "1000"
        )

        # Initialize variables for pagination
        nextPageToken <- NULL
        all_studies <- list()

        repeat {
        # Update the payload with the nextPageToken if it exists
        if (!is.null(nextPageToken)) {
            payload$pageToken <- nextPageToken
        }
        
        # Make the API request
        response <- request(base_url) |>
            req_method("GET") |>
            req_url_query(!!!payload) |>
            req_perform()
        
        # Parse the response
        res <- resp_body_json(response)
        
        # Check if there are studies in the response and append them
        if (length(res$studies) > 0) {
            all_studies <- c(all_studies, res$studies)
        } else {
            # If no studies, exit the loop
            break
        }
        
        # Update the nextPageToken for the next request
        nextPageToken <- res$nextPageToken
        
        # Break the loop if nextPageToken is null or empty, indicating no more pages
        if (is.null(nextPageToken) || nextPageToken == "") {
            break
  }
}

# Convert the list of studies to a data frame
df <- purrr::map_dfr(all_studies, ~as.data.frame(.x))
# Print out the total number of records fetched
print(paste("Total records fetched:", nrow(df)))

return(df)
}